<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluate RAG with LLM Evals</h1>

In this tutorial we will look into building a RAG pipeline and evaluating it with Phoenix Evals.

It has the the following sections:

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG (with the help of a framework such as LlamaIndex).
3. Evaluating RAG with Phoenix Evals.

## Retrieval Augmented Generation (RAG)

LLMs are trained on vast datasets, but these will not include your specific data (things like company knowledge bases and documentation). Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data as context during the generation process. This is done not by altering the training data of the LLMs but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and prepared for queries. This process is called indexing. User queries act on this index, which filters your data down to the most relevant context. This context and your query then are sent to the LLM along with a prompt, and the LLM provides a response.

RAG is a critical component for building applications such a chatbots or agents and you will want to know RAG techniques on how to get data into your application.

<img src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/RAG_Pipeline.png">

## Stages within RAG

There are five key stages within RAG, which will in turn be a part of any larger RAG application.

- **Loading**: This refers to getting your data from where it lives - whether it's text files, PDFs, another website, a database or an API - into your pipeline.
- **Indexing**: This means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.
- **Storing**: Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

- **Querying**: For any given indexing strategy there are many ways you can utilize LLMs and data structures to query, including sub-queries, multi-step queries, and hybrid strategies. 
- **Evaluation**: A critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures on how accurate, faithful, and fast your responses to queries are.


## Build a RAG system 

Now that we have understood the stages of RAG, let's build a pipeline. We will use [LlamaIndex](https://www.llamaindex.ai/) for RAG and [Phoenix Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) for evaluation.


In [1]:
!pip install -qq "arize-phoenix[experimental,llama-index]>=2.0"

In [1]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

import os
from getpass import getpass

import pandas as pd
import phoenix as px
from llama_index import SimpleDirectoryReader, VectorStoreIndex, set_global_handler
from llama_index.llms import OpenAI
from llama_index.node_parser import SimpleNodeParser

During this tutorial, we will capture all the data we need to evaluate our RAG pipeline using Phoenix Tracing. To enable this, simply start the phoenix application and instrument LlamaIndex.

In [2]:
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
set_global_handler("arize_phoenix")

For this tutorial we will be using OpenAI for creating synthetic data as well as for evaluation. 

In [4]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Let's use an [essay by Paul Graham](https://www.paulgraham.com/worked.html) to build our RAG pipeline.

In [5]:
!mkdir -p 'data/paul_graham/'
!curl 'https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt' -o 'data/paul_graham/paul_graham_essay.txt'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 75041  100 75041    0     0   132k      0 --:--:-- --:--:-- --:--:--  133k


### Load Data and Build an Index

In [6]:
documents = SimpleDirectoryReader("./data/paul_graham/").load_data()

# Define an LLM
llm = OpenAI(model="gpt-4")

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [7]:
query_engine = vector_index.as_query_engine()

In [8]:
response_vector = query_engine.query("What did the author do growing up?")

Check the response that you get from the query.

In [9]:
response_vector.response

"The author, growing up, worked on writing and programming. They wrote short stories and tried writing programs on an IBM 1401 computer. They used an early version of Fortran and had to type programs on punch cards. However, they were puzzled by the computer and didn't have much data to work with. The author also mentioned that with the advent of microcomputers, everything changed."

By default LlamaIndex retrieves two similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [10]:
# First retrieved node
response_vector.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [11]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"It felt like I was doing life right. I remember that because I was slightly dismayed at how novel it felt. The good news is that I had more moments like this over the next few years.\n\nIn the summer of 2016 we moved to England. We wanted our kids to see what it was like living in another country, and since I was a British citizen by birth, that seemed the obvious choice. We only meant to stay for a year, but we liked it so much that we still live there. So most of Bel was written in England.\n\nIn the fall of 2019, Bel was finally finished. Like McCarthy's original Lisp, it's a spec rather than an implementation, although like McCarthy's Lisp it's a spec expressed as code.\n\nNow that I could write essays again, I wrote a bunch about topics I'd had stacked up. I kept writing essays through 2020, but I also started to think about other things I could work on. How should I choose what to do? Well, how had I chosen what to work on in the past? I wrote an essay for myself to answer that 

Remember that we are using Phoenix Tracing to capture all the data we need to evaluate our RAG pipeline. You can view the traces in the phoenix application.

In [12]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


We can access the traces by directly pulling the spans from the phoenix session.

In [13]:
spans_df = px.active_session().get_spans_dataframe()

In [14]:
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

,name,span_kind,attributes.input.value,attributes.retrieval.documents
context.span_id,,,,
f16f494c-e0de-4390-8350-4ed99b7d2449,llm,LLM,NaN,NaN
2c450026-7776-4872-a786-9e1b0d68c070,synthesize,CHAIN,What did the author do growing up?,NaN
de12ae5a-d351-4e15-9239-89356e8e5aae,embedding,EMBEDDING,NaN,NaN
f52573ad-2d39-4352-9540-4577fcf8e244,retrieve,RETRIEVER,What did the author do growing up?,[{'document.content': 'What I Worked On Febru...
4989e6ef-ee31-4b12-ae4b-1479f9697b83,query,CHAIN,What did the author do growing up?,NaN


Note that the traces have captured the documents that were retrieved by the query engine. This is nice because it means we can introspect the documents without having to keep track of them ourselves.

In [15]:
spans_with_docs_df = spans_df[spans_df["attributes.retrieval.documents"].notnull()]

In [16]:
spans_with_docs_df[["attributes.input.value", "attributes.retrieval.documents"]].head()

,attributes.input.value,attributes.retrieval.documents
context.span_id,,
f52573ad-2d39-4352-9540-4577fcf8e244,What did the author do growing up?,[{'document.content': 'What I Worked On Febru...


We have built a RAG pipeline and also have instrumented it using Phoenix Tracing. We now need to evaluate it's performance. We can assess our RAG system/query engine using Phoenix's LLM Evals. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

- **Retrieval Evaluation**: To assess the accuracy and relevance of the documents that were retrieved
- **Response Evaluation**: Measure the appropriateness of the response generated by the system when the context was provided.

### Generate Question Context Pairs

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response.

For this tutorial, let's use Phoenix's `llm_generate` to help us create the question-context pairs.

First, let's create a dataframe of all the document chunks that we have indexed.

In [17]:
# Let's construct a dataframe of just the documents that are in our index
document_chunks_df = pd.DataFrame({"text": [node.get_text() for node in nodes]})
document_chunks_df.head()

,text
0,What I Worked On\n\nFebruary 2021\n\nBefore co...
1,I was puzzled by the 1401. I couldn't figure o...
2,I remember vividly how impressed and envious I...
3,I couldn't have put this into words when I was...
4,This was more like it; this was what I had exp...


Now that we have the document chunks, let's prompt an LLM to generate us 3 questions per chunk. Note that you could manually solicit questions from your team or customers, but this is a quick and easy way to generate a large number of questions.

In [18]:
generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [19]:
import json

from phoenix.experimental.evals import OpenAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


questions_df = llm_generate(
    dataframe=document_chunks_df,
    template=generate_questions_template,
    model=OpenAIModel(
        model_name="gpt-3.5-turbo",
    ),
    output_parser=output_parser,
    concurrency=20,
)

llm_generate |          | 0/58 (0.0%) | ⏳ 00:00<? | ?it/s

In [20]:
questions_df.head()

,question_1,question_2,question_3
0,What were the two main things the author worke...,What was the language used for programming on ...,What was the author's clearest memory regardin...
1,What were the limitations of the 1401 computer...,How did microcomputers change the author's exp...,Why did the author's father buy a TRS-80 compu...
2,What was the author's first experience with co...,Why did the author decide to switch from study...,What were the two things that influenced the a...
3,What were the two things that inspired the aut...,What programming language did the author learn...,What was the author's undergraduate thesis about?
4,What was the author's undergraduate thesis about?,Which three grad schools did the author apply to?,What realization did the author have during th...


In [21]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

The LLM has generated three questions per chunk. Let's take a quick look.

In [22]:
questions_with_document_chunk_df.head(10)

,text,question
0,What I Worked On\n\nFebruary 2021\n\nBefore co...,What were the two main things the author worke...
1,I was puzzled by the 1401. I couldn't figure o...,What were the limitations of the 1401 computer...
2,I remember vividly how impressed and envious I...,What was the author's first experience with co...
3,I couldn't have put this into words when I was...,What were the two things that inspired the aut...
4,This was more like it; this was what I had exp...,What was the author's undergraduate thesis about?
5,"Only Harvard accepted me, so that was where I ...",What realization did the author have during th...
6,"So I decided to focus on Lisp. In fact, I deci...",What motivated the author to write a book abou...
7,Anyone who wanted one to play around with coul...,What was the author's initial hesitation in ge...
8,I knew intellectually that people made art — t...,What was the author's initial perception of pe...
9,Then one day in April 1990 a crack appeared in...,What was the author's initial plan for their d...


### Retrieval Evaluation

We are now prepared to perform our retrieval evaluations. We will execute the queries we generated in the previous step and verify whether or not that the correct context is retrieved.

In [23]:
# First things first, let's reset phoenix
px.close_app()
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [24]:
# loop over the questions and generate the answers
for _, row in questions_with_document_chunk_df.iterrows():
    question = row["question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Question: What were the two main things the author worked on before college?
Answer: The author worked on writing and programming before college.

Question: What were the limitations of the 1401 computer and why couldn't the author do much with it?
Answer: The author mentions that the 1401 computer had limitations in terms of input. The only form of input to programs was data stored on punched cards, and the author did not have any data stored on punched cards. Additionally, the author did not know enough math to do anything interesting that did not rely on input. Therefore, the author couldn't do much with the 1401 computer due to these limitations.

Question: What was the author's first experience with computers and programming?
Answer: The author's first experience with computers and programming was in 9th grade when they had access to an IBM 1401 computer in the basement of their junior high school. They used an early version of Fortran and had to type programs on punch cards. Howe

KeyboardInterrupt: 

Now that we have executed the queries, we can start validating whether or not the RAG system was able to retrieve the correct context.

In [31]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.active_session())
retrieved_documents_df

context.trace_id  \
context.span_id                      document_position                                         
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  b5078a89-dabf-4275-a977-895cde97caa5   
                                     1                  b5078a89-dabf-4275-a977-895cde97caa5   
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  a096384e-555e-40f3-ba74-60159540b6eb   
                                     1                  a096384e-555e-40f3-ba74-60159540b6eb   
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                  95624502-fdd3-4b29-ab4c-6667d3007e8d   
...                                                                                      ...   
1ae5c178-5629-4af8-b0ff-95a665b7a8d6 1                  03023b7f-e08f-4f9d-83d4-bb90d93f62c5   
05884bf5-2516-46dc-805f-6f53999be2f5 0                  c8381134-82c7-4957-b1c3-1290242d941c   
                                     1                  c8381134-82c7-4957-b1c3-1290242d941c   
324293df-d675-472d-8956-817b896a2169 0                  574de370-7312-41f7-b19e-c3bb1c6ff98c   
                                     1                  574de370-7312-41f7-b19e-c3bb1c6ff98c   

                                                                                                    input  \
context.span_id                      document_position                                                      
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  Where did the author and Robert work on their ...   
                                     1                  Where did the author and Robert work on their ...   
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  What was the author's startup idea and why was...   
                                     1                  What was the author's startup idea and why was...   
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                  Who were Idelle and Julian Weber and what was ...   
...                                                                                                   ...   
1ae5c178-5629-4af8-b0ff-95a665b7a8d6 1                  What was the author's first experience with co...   
05884bf5-2516-46dc-805f-6f53999be2f5 0                  What were the limitations of the 1401 computer...   
                                     1                  What were the limitations of the 1401 computer...   
324293df-d675-472d-8956-817b896a2169 0                  What were the two main things the author worke...   
                                     1                  What were the two main things the author worke...   

                                                                                                reference  \
context.span_id                      document_position                                                      
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  It may look clunky today, but in 1996 it was t...   
                                     1                  Ten years later this deal became the model for...   
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  One night in October 2003 there was a big part...   
                                     1                  One of my tricks for writing essays had always...   
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                  So with my unerring nose for financial opportu...   
...                                                                                                   ...   
1ae5c178-5629-4af8-b0ff-95a665b7a8d6 1                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
05884bf5-2516-46dc-805f-6f53999be2f5 0                  I was puzzled by the 1401. I couldn't figure o...   
                                     1                  I remember vividly how impressed and envious I...   
324293df-d675-472d-8956-817b896a2169 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                                     1                  Then one day in April 1990 a crack appeared i

Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [32]:
from phoenix.experimental.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(OpenAIModel(model_name="gpt-4-1106-preview"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

run_evals |          | 0/274 (0.0%) | ⏳ 00:00<? | ?it/s

Failed attempt 1: raised InternalServerError("Error code: 500 - {'error': {'message': 'Request failed due to server shutdown', 'type': 'server_error', 'param': None, 'code': None}}")


In [34]:
retrieved_documents_relevance_df.head()

label  score  \
context.span_id                      document_position                     
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  unrelated      0   
                                     1                  unrelated      0   
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  unrelated      0   
                                     1                   relevant      1   
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                   relevant      1   

                                                                                              explanation  
context.span_id                      document_position                                                     
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  The reference text does not provide specific i...  
                                     1                  The question asks for specific information abo...  
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  The question asks for two specific pieces of i...  
                                     1                  The question asks for two specific pieces of i...  
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                  The reference text provides information about ...

We can now combine the documents with the relevance evaluations to compute retrieval metrics. These metrics will help us understand how well the RAG system is performing.

In [35]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

context.trace_id  \
context.span_id                      document_position                                         
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  b5078a89-dabf-4275-a977-895cde97caa5   
                                     1                  b5078a89-dabf-4275-a977-895cde97caa5   
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  a096384e-555e-40f3-ba74-60159540b6eb   
                                     1                  a096384e-555e-40f3-ba74-60159540b6eb   
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                  95624502-fdd3-4b29-ab4c-6667d3007e8d   
...                                                                                      ...   
1ae5c178-5629-4af8-b0ff-95a665b7a8d6 1                  03023b7f-e08f-4f9d-83d4-bb90d93f62c5   
05884bf5-2516-46dc-805f-6f53999be2f5 0                  c8381134-82c7-4957-b1c3-1290242d941c   
                                     1                  c8381134-82c7-4957-b1c3-1290242d941c   
324293df-d675-472d-8956-817b896a2169 0                  574de370-7312-41f7-b19e-c3bb1c6ff98c   
                                     1                  574de370-7312-41f7-b19e-c3bb1c6ff98c   

                                                                                                    input  \
context.span_id                      document_position                                                      
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  Where did the author and Robert work on their ...   
                                     1                  Where did the author and Robert work on their ...   
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  What was the author's startup idea and why was...   
                                     1                  What was the author's startup idea and why was...   
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                  Who were Idelle and Julian Weber and what was ...   
...                                                                                                   ...   
1ae5c178-5629-4af8-b0ff-95a665b7a8d6 1                  What was the author's first experience with co...   
05884bf5-2516-46dc-805f-6f53999be2f5 0                  What were the limitations of the 1401 computer...   
                                     1                  What were the limitations of the 1401 computer...   
324293df-d675-472d-8956-817b896a2169 0                  What were the two main things the author worke...   
                                     1                  What were the two main things the author worke...   

                                                                                                reference  \
context.span_id                      document_position                                                      
9ca8e2d3-2c2c-4ff9-908c-522d05304a88 0                  It may look clunky today, but in 1996 it was t...   
                                     1                  Ten years later this deal became the model for...   
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef 0                  One night in October 2003 there was a big part...   
                                     1                  One of my tricks for writing essays had always...   
f015b2f9-0bac-4b87-85a6-1f9efd71c622 0                  So with my unerring nose for financial opportu...   
...                                                                                                   ...   
1ae5c178-5629-4af8-b0ff-95a665b7a8d6 1                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
05884bf5-2516-46dc-805f-6f53999be2f5 0                  I was puzzled by the 1401. I couldn't figure o...   
                                     1                  I remember vividly how impressed and envious I...   
324293df-d675-472d-8956-817b896a2169 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                                     1                  Then one day in April 1990 a crack appeared i

Let's compute Normalized Discounted Cumulative Gain [NCDG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) at 2 for all our retrieval steps.  In information retrieval, this metric is often used to measure effectiveness of search engine algorithms and related applications.

In [36]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [37]:
ndcg_at_2

,score
context.span_id,
005828a2-0216-4e58-9f08-10f8cccbc5f2,1.0
013b7ba2-2664-4ac5-b21f-7e55f05dfeea,1.0
05884bf5-2516-46dc-805f-6f53999be2f5,1.0
06201524-0733-408c-8462-760f03ef7c3d,1.0
09d1a8d9-a31c-449e-9c62-3431bb10636f,1.0
...,...
f8eac362-d652-4dac-b0e8-e6cfc66d4856,1.0
fad84e31-5c09-47ca-af03-4aac70914099,1.0
fc6a536b-84dc-4dff-ba86-abc05351335c,0.0


Let's also compute precision at 2 for all our retrieval steps.

In [38]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [39]:
precision_at_2

,score
context.span_id,
005828a2-0216-4e58-9f08-10f8cccbc5f2,1.0
013b7ba2-2664-4ac5-b21f-7e55f05dfeea,1.0
05884bf5-2516-46dc-805f-6f53999be2f5,0.5
06201524-0733-408c-8462-760f03ef7c3d,1.0
09d1a8d9-a31c-449e-9c62-3431bb10636f,1.0
...,...
f8eac362-d652-4dac-b0e8-e6cfc66d4856,1.0
fad84e31-5c09-47ca-af03-4aac70914099,1.0
fc6a536b-84dc-4dff-ba86-abc05351335c,0.0


Lastly, let's compute whether or not a correct document was retrieved at all for each query (e.g. a hit)

In [57]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

Let's now view the results in a combined dataframe.

In [58]:
retrievals_df = px.active_session().get_spans_dataframe("span_kind == 'RETRIEVER'")
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        ndcg_at_2.add_prefix("ncdg@2_"),
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

,attributes.input.value,ncdg@2_score,precision@2_score,hit
context.span_id,,,,
9ca8e2d3-2c2c-4ff9-908c-522d05304a88,Where did the author and Robert work on their ...,0.00000,0.0,False
c093ef44-b8b9-4c1f-8284-b0f24c5bdcef,What was the author's startup idea and why was...,0.63093,0.5,True
f015b2f9-0bac-4b87-85a6-1f9efd71c622,Who were Idelle and Julian Weber and what was ...,1.00000,1.0,True
0f55de9d-01d0-43d6-95ac-036a88c0a738,Why did the author drop out of RISD in 1993?,1.00000,0.5,True
652943b0-9cf5-4dad-8697-19be615efe11,"In the context of art school, what is a 'signa...",1.00000,1.0,True
...,...,...,...,...
170eaefd-5699-4480-b989-e17a8254f11b,What was the author's undergraduate thesis about?,0.00000,0.0,False
379fe37f-2f86-42b8-aa7a-dbadf403e6f8,What were the two things that inspired the aut...,1.00000,1.0,True
1ae5c178-5629-4af8-b0ff-95a665b7a8d6,What was the author's first experience with co...,1.00000,1.0,True


### Observations

Let's now take our results and aggregate them to get a sense of how well our RAG system is performing.

In [59]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

ncdg@2_score         0.920054
precision@2_score    0.813869
hit                  0.941606
dtype: float64

As we can see from the above numbers, our RAG system is not perfect, there are times when it fails to retrieve the correct context within the first two documents. At other times the correct context is included in the top 2 results but non-relevant information is also included in the context. This is an indication that we need to improve our retrieval strategy. One possible solution could be to increase the number of documents retrieved and then use a more sophisticated ranking strategy (such as a reranker) to select the correct context.

We have now evaluated our RAG system's retrieval performance. Let's send these evaluations to Phoenix for visualization. By sending the evaluations to Phoenix, you will be able to view the evaluations alongside the traces that were captured earlier.

In [60]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

Sending Evaluations: 100%|██████████| 548/548 [00:01<00:00, 407.23it/s]


### Response Evaluation

The retrieval evaluations demonstrates that our RAG system is not perfect. However, it's possible that the LLM is able to generate the correct response even when the context is incorrect. Let's evaluate the responses generated by the LLM.

In [43]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.active_session())
qa_with_reference_df

,input,output,reference
context.span_id,,,
943e4328-7594-4f5e-ae9c-8068546f97ad,Where did the author and Robert work on their ...,None,"It may look clunky today, but in 1996 it was t..."
4b685ee1-89f1-40c0-b61c-8815a5bb0453,What was the author's startup idea and why was...,The author's startup idea was to start their o...,One night in October 2003 there was a big part...
231363ee-5991-4f4b-8f47-d33321d2e087,Who were Idelle and Julian Weber and what was ...,Idelle and Julian Weber were individuals who w...,So with my unerring nose for financial opportu...
977ada2c-4e56-4c67-a93a-c282b96deb98,Why did the author drop out of RISD in 1993?,The context information does not provide any i...,"This was not as strange as it sounds, because ..."
77f66c4e-1649-460a-a2e2-955d2bab34b7,"In the context of art school, what is a 'signa...",A 'signature style' in the context of art scho...,"But the most important thing I learned, and wh..."
...,...,...,...
5c0a5318-61db-494c-b3e3-512421c9cd89,What was the author's undergraduate thesis about?,The context information does not provide any i...,I knew intellectually that people made art — t...
0467e172-d2c7-46ed-9f62-b2ad83d66fa9,What were the two things that inspired the aut...,The two things that inspired the author to wor...,"Only Harvard accepted me, so that was where I ..."
a01a4a8e-86eb-4f78-ad54-497bdab55717,What was the author's first experience with co...,The author's first experience with computers a...,I remember vividly how impressed and envious I...


Now that we have a dataset of the question, context, and response (input, reference, and output), we now can measure how well the LLM is responding to the queries. For details on the QA correctness evaluation, see the [LLM Evals documentation](https://docs.arize.com/phoenix/llm-evals/running-pre-tested-evals/q-and-a-on-retrieved-data).

In [46]:
from phoenix.experimental.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(OpenAIModel(model_name="gpt-4-1106-preview"))
hallucination_evaluator = HallucinationEvaluator(OpenAIModel(model_name="gpt-4-1106-preview"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/274 (0.0%) | ⏳ 00:00<? | ?it/s

In [47]:
qa_correctness_eval_df.head()

,label,score,explanation
context.span_id,,,
943e4328-7594-4f5e-ae9c-8068546f97ad,correct,1.0,The question asks for specific information abo...
4b685ee1-89f1-40c0-b61c-8815a5bb0453,incorrect,0.0,The reference text describes the author's idea...
231363ee-5991-4f4b-8f47-d33321d2e087,incorrect,0.0,The question asks for the identity of Idelle a...
977ada2c-4e56-4c67-a93a-c282b96deb98,correct,1.0,"To determine if the answer is correct, we must..."
77f66c4e-1649-460a-a2e2-955d2bab34b7,correct,1.0,"To determine if the answer is correct, we must..."


In [48]:
hallucination_eval_df.head()

,label,score,explanation
context.span_id,,,
943e4328-7594-4f5e-ae9c-8068546f97ad,factual,0.0,The query asks for specific information about ...
4b685ee1-89f1-40c0-b61c-8815a5bb0453,hallucinated,1.0,The reference text describes the author's idea...
231363ee-5991-4f4b-8f47-d33321d2e087,hallucinated,1.0,The reference text provides information about ...
977ada2c-4e56-4c67-a93a-c282b96deb98,factual,0.0,The query asks for the reason why the author d...
77f66c4e-1649-460a-a2e2-955d2bab34b7,factual,0.0,The answer provided aligns with the informatio...


#### Observations

Let's now take our results and aggregate them to get a sense of how well the LLM is answering the questions given the context.

In [49]:
qa_correctness_eval_df.mean(numeric_only=True)

score    0.890511
dtype: float64

In [50]:
hallucination_eval_df.mean(numeric_only=True)

score    0.080292
dtype: float64

Our QA Correctness score of `0.91` and a Hallucinations score `0.1` (hallucinations exist) signifies that the generated answers are correct ~91% of the time - there is room for improvement. This could be due to the retrieval strategy or the LLM itself. We will need to investigate further to determine the root cause.

Since we have evaluated our RAG system's QA performance, let's send these evaluations to Phoenix for visualization.

In [51]:
from phoenix.trace import SpanEvaluations

px.log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

Sending Evaluations: 100%|██████████| 274/274 [00:00<00:00, 439.61it/s]


We now have sent all our evaluations to Phoenix. Let's go to the Phoenix application and view the results! Since we've sent all the evals to Phoenix, we can analyze the results together to make a determination on whether or not poor retrieval or irrelevant context has an effect on the LLM's ability to generate the correct response.

In [49]:
print("phoenix URL", px.active_session().url)

phoenix URL http://127.0.0.1:6006/


## Conclusion

We have explored how to build and evaluate a RAG pipeline using LlamaIndex and Phoenix, with a specific focus on evaluating the retrieval system and generated responses within the pipelines. 

Phoenix offers a variety of other evaluations that can be used to assess the performance of your LLM Application. For more details, see the [LLM Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) documentation.